# Test Cubeviz Base Functionality with Glupyter

This work is to satisfy https://github.com/spacetelescope/dat_pyinthesky/issues/28. The goals are:

1. Load a data cube
2. Display 1D and 2D
3. Median collapse cube
4. Stretch goal: Hover stats with X, Y, value, RA, DEC

This notebook was originally written by Pey-Lian Lim and updated by Tom Robitaille.

In [ ]:
import numpy as np
import glue_jupyter as gj

### Step 1 - load a data cube

Start off by downloading the required data

In [ ]:
import os
from urllib.request import urlretrieve
if not os.path.exists('manga-7495-12704-LOGCUBE.fits.gz'):
    _ = urlretrieve('http://data.sdss.org/sas/dr13/manga/spectro/redux/v1_5_4/7495/stack/manga-7495-12704-LOGCUBE.fits.gz',
                    'manga-7495-12704-LOGCUBE.fits.gz')

Next we start up glue then load the data in

In [ ]:
app = gj.jglue()

In [ ]:
flux, ivar, mask = app.load_data('manga-7495-12704-LOGCUBE.fits')[0:3]

In [ ]:
app.data_collection.merge(flux, ivar, mask, label='manga-cube')

In [ ]:
cube = app.data_collection[14]
print(cube)

### Step 2 - Show 2D image and 1D spectrum view

In [ ]:
image = app.imshow(data=cube)

In [ ]:
profile = app.profile1d(data=cube)

### Step 3 - Median collapse spectral range

Before continuing, select a spectral range in the profile viewer, preferably around an interesting line!

In [ ]:
from glue.core import Data
data_subset = cube['FLUX'].copy()
data_subset[~cube.subsets[0].to_mask()] = np.nan
data_subset_collapsed = np.nanmedian(data_subset, axis=0)
cube_subset = Data(FLUX=data_subset_collapsed, label='Median collapsed')

In [ ]:
app.add_data(cube_subset)

In [ ]:
image_collapsed = app.imshow(data=cube_subset)

### Step 4 - show hover over 

For now, this requires manual code, but we could incorporate this into glue-jupyter

In [ ]:
from ipyevents import Event
from ipywidgets import Text, VBox

t = Text()
d = Event(source=image_collapsed.figure, watched_events=['mousemove'])
def handle_event(event):
    x = event['relativeX']
    y = event['relativeX']
    t.value = 'x={0} y={1}'.format(x, y)
d.on_dom_event(handle_event)

image_collapsed.main_widget.children += (t,)

This adds a small box with the x/y coordinates below the collapsed image viewer. Note that for now the coordinates are not actually the data coordinates, but with some upcoming changes in ipyevents we will be able to convert these to real pixel coordinates and subsequently world coordinates.